# Converting saves

In [1]:
from forgebox.imports import *
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [2]:
WEIGHTS = Path("/nvme/GCI/public/lit/weights")

In [3]:
tag = "roberta-large"

In [4]:
SAVE = WEIGHTS/"rbt_lg_0725_155121"

In [5]:
def save_configs(tag, save):
    config = AutoConfig.from_pretrained(tag)
    tokenizer = AutoTokenizer.from_pretrained(tag)
    tokenizer.save_pretrained(save/"tokenizer")
    config.save_pretrained(save/"config")
    return tokenizer, config

In [6]:
tokenizer, config = save_configs(tag, SAVE)

In [7]:
WEIGHTS.ls()

['bert_cased_0724_193954',
 'rbt_lg_0725_155121',
 'rbt_lg_finer_0725_160941',
 'bert_buc_0724_194410',
 'bert_buc_0725_002308',
 'bert_0724_193402',
 'bert_0724_231826']

In [11]:
class LitLM(nn.Module):
    def __init__(self, base):
        super().__init__()
        self.base =  base
        self.learning_rate=1e-3
        self.config = self.base.config
        self.reg = nn.Linear(base.config.hidden_size, 1)
        self.crit = nn.MSELoss()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        vec  =self.base(x).last_hidden_state[:,0,:]
        return self.reg(vec)
    
    def inference(
        lit_model, model_weights:Path, data_loader:DataLoader, filename:str
    ) -> None:
        lit_model.load_state_dict(torch.load(str(model_weights)), strict=False)
        lit_model = lit_model.eval()
        lit_model = lit_model.cuda()
        results = []
        for batch_idx, batch in tqdm(enumerate(data_loader), leave=False):
            ids = batch["id"]
            x = batch["excerpt"].cuda()
            with torch.no_grad():
                y_ = lit_model(x)[:,0].detach().cpu().numpy()
            results.append(pd.DataFrame({"id":ids, "target":y_}))
        pd.concat(results).to_csv(filename, index=False)
        lit_model.cpu()

In [9]:
for ckpt in list(i for i in SAVE.iterdir() if i.name[-5:]==".ckpt"):
    state = torch.load(ckpt, map_location='cpu')['state_dict']
    torch.save(state, ckpt.parent/(ckpt.name.replace("=","-").replace(".ckpt",".h5")))

In [12]:
LitLM(AutoModel.from_config(config)).load_state_dict(torch.load(ckpt, map_location='cpu')['state_dict'])

<All keys matched successfully>

In [13]:
for i in SAVE.iterdir():
    if i.name[-5:]==".ckpt":
        os.system(f"rm -f {i}")
        
!ls {SAVE}

config			      epoch-20-val_loss-0.27fd1.h5  tokenizer
epoch-10-val_loss-0.26fd0.h5  epoch-5-val_loss-0.30fd2.h5
epoch-11-val_loss-0.31fd3.h5  epoch-9-val_loss-0.25fd4.h5


In [14]:
!kaggle datasets init -p {SAVE}

Data package template written to: /nvme/GCI/public/lit/weights/rbt_lg_0725_155121/dataset-metadata.json


In [15]:
!cat {SAVE/"dataset-metadata.json"}

{
  "title": "INSERT_TITLE_HERE",
  "id": "raynardj/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [16]:
%%writefile {SAVE/"dataset-metadata.json"}
{
  "title": "rbt_lg_0725_155121",
  "id": "raynardj/rbt-lg-0725-155121",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

Overwriting /nvme/GCI/public/lit/weights/rbt_lg_0725_155121/dataset-metadata.json


In [17]:
!kaggle datasets create -r tar -u -p {SAVE}

Starting upload for file tokenizer.tar
100%|███████████████████████████████████████| 2.50M/2.50M [00:05<00:00, 474kB/s]
Upload successful: tokenizer.tar (2MB)
Starting upload for file epoch-9-val_loss-0.25fd4.h5
100%|██████████████████████████████████████| 1.32G/1.32G [20:25<00:00, 1.16MB/s]
Upload successful: epoch-9-val_loss-0.25fd4.h5 (1GB)
Starting upload for file config.tar
100%|██████████████████████████████████████| 10.0k/10.0k [00:03<00:00, 2.65kB/s]
Upload successful: config.tar (10KB)
Starting upload for file epoch-10-val_loss-0.26fd0.h5
100%|██████████████████████████████████████| 1.32G/1.32G [20:27<00:00, 1.16MB/s]
Upload successful: epoch-10-val_loss-0.26fd0.h5 (1GB)
Starting upload for file epoch-20-val_loss-0.27fd1.h5
100%|██████████████████████████████████████| 1.32G/1.32G [20:40<00:00, 1.15MB/s]
Upload successful: epoch-20-val_loss-0.27fd1.h5 (1GB)
Starting upload for file epoch-11-val_loss-0.31fd3.h5
100%|██████████████████████████████████████| 1.32G/1.32G [20:20<00:0

In [62]:
!ls -R {SAVE}

/nvme/GCI/public/lit/weights/bert_0724_231826:
config			      epoch-15-val_loss-0.30fd2.h5
dataset-metadata.json	      epoch-16-val_loss-0.28fd0.h5
epoch-10-val_loss-0.27fd4.h5  epoch-28-val_loss-0.29fd3.h5
epoch-10-val_loss-0.31fd1.h5  tokenizer

/nvme/GCI/public/lit/weights/bert_0724_231826/config:
config.json

/nvme/GCI/public/lit/weights/bert_0724_231826/tokenizer:
special_tokens_map.json  tokenizer_config.json	tokenizer.json	vocab.txt
